# CONFPASS: Conformer Prioritizations & Analysis for DFT reoptimisations
## User guide // Tutorial (Jupyter Notebook Version 3.0) 

◑▽◐	｡◕‿◕｡	◕‿◕	◕‿- ◑▽◐	｡◕‿◕｡	◕‿◕	◕‿- ◑▽◐	｡◕‿◕｡	◕‿◕	◕‿-

First Created on Wed Jul  6 10:24:05 2022

Last updated on  Aug 15 2022

@author: Ching Ching Lam 

◑▽◐	｡◕‿◕｡	◕‿◕	◕‿- ◑▽◐	｡◕‿◕｡	◕‿◕	◕‿- ◑▽◐	｡◕‿◕｡	◕‿◕	◕‿-

This package is suitable for processing organic molecules, including radical species (excluding organometallics and inorganic compounds).

There are two parts in this package:

PART 1: class conp (pre-reoptimisations: clustering and priority list generation) 

PART 2: class pas (post-reoptimisations: predict the completeness of the reoptimisation process) Random forest (RF) models are used for making the predictions. The specific setting for the RF model is {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 50}.

The package is written entirely in python and developed under the following environment:

-	The Python (3.8.12) Standard Library (os, sys, itertools, collections, pickle, random, optparse, traceback)
-	Pandas (1.3.4)
-	Numpy (1.21.2)
-	Sklearn (1.0.1)
-	RDkit (2019.09.3)
-   Natsort (8.0.2)


Let's start by importing the module:

In [1]:
from confpass import confpass 

RDKit WARNING: [11:17:24] Enabling RDKit 2019.09.3 jupyter extensions
[11:17:24] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
confpass.__version__

'v15082022'

### PART 1: class conp (pre-reoptimisations: clustering and priority list generation) 

## class confpass.conp

### __init__(self, sdf_files) 

#### Parameters 

sdf_files: the path of the sdf file 

### confpass.conp.get_priority(self, x=0.8, x_de=0.2, n=3, method = 'pipe_x_as')

#### Parameters 

method: string; priority list assembling method; choices=('pipe_x_as', 'pipe_x','pipe_as','nth','random','ascend')

x: float; hyperparameter for the pipe_x method, default = 0.8; i.e. use the clustering result when n_clusters=total_conformer_no*x

x_de: float; hyperparameter for pipe_x_as method, default = 0.2; i.e. 20% (pipe_x priority list) + 80%(pipe_de priority list)

-n: float; hyperparameter for nth method, default = 3; i.e. prioritise every 3rd conformer

#### Attributes

methodf: string; the choice of method as specified during execution 

clustering_result: pd.DataFrame; data frame of the clustering result (the conformer index starts from 0)

priority_df: pd.DataFrame; priority list data frame (the conformer index starts from 0)

reidx_priority_df: pd.DataFrame; priority list data frame (the conformer index starts from 1)


### confpass.conp.priority2csv(self)

Should only be applied after conp.get_priority(); create a .csv version of the reindexed priority list data frame (conp.reidx_priority_df; the conformer index starts from 1)

#### No parameters or attributes

### confpass.conp.priority2gjf(self, keywords, space, per, radical = False, rmAtom_ls=[])

#### Parameters 

keywords: string; text in the gjf file for Gaussian calculations located before the xyz coordinates 

space: string; text in the gjf file for Gaussian calculations located after the xyz coordinates 

per: float; hyperparameter for priority2gjf() function; percentage of the conformers to be converted into gjf in the priority list

radical: bool; default = False; need to change to True when processing radicals - with a different number of atoms compared to the pseudo structure

rmAtom_ls: list; specify to remove atoms from the pseudo structure when generating the gjf file; i.e. atom index starts from 1 


#### Attributes

None

### confpass.conp.get_gjf(self, keywords, space, conf_idx_ls, radical = False, rmAtom_ls=[])

#### Parameters 

keywords: string; text in the gjf file for Gaussian calculations located before the xyz coordinates 

space: string; text in the gjf file for Gaussian calculations located after the xyz coordinates 

conf_idx_ls: list of conformer indexes (the conformer index starts from 1)

radical: bool; default = False; need to change to True when processing radicals - with a different number of atoms compared to the pseudo structure

rmAtom_ls: list; specify to remove atoms from the pseudo structure when generating the gjf file; i.e. atom index starts from 1 

#### Attributes

None

## Examples

In [3]:
## 1. Generate the priority list for a sdf file from a conformational searching calculation with 
## the default setting -- x=0.8, x_de=0.2, method = 'pipe_x_de'

test1=confpass.conp(['test_15.sdf'])
test1.get_priority()

## for conp.priority_df, the index of the conformer starts from 0
test1.priority_df

idx         name                                        priority_ls
0    0  test_15.sdf  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 16, 23,...

In [4]:
## for reidx_priority_df, the index of the conformer starts from 1 
test1.reidx_priority_df

name                                        priority_ls
0  test_15.sdf  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 20, 17, 24...

In [5]:
## check out the clustering result (before the priority generation process); 
## here the index of the conformer starts from 0
test1.clustering_result.head(5)

name  n     nor_n                                           clusters
0  test_15.sdf  1  0.018519  [[0, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12...
1  test_15.sdf  2  0.037037  [[0, [0, 1, 2, 4, 5, 11, 13, 15, 16, 17, 19, 2...
2  test_15.sdf  3  0.055556  [[2, [0, 4, 5, 11, 16, 17, 25, 27, 29, 34, 38,...
3  test_15.sdf  4  0.074074  [[2, [0, 4, 5, 11, 16, 17, 25, 27, 29, 34, 38,...
4  test_15.sdf  5  0.092593  [[0, [0, 4, 5, 11, 16, 17, 25, 27, 29, 34, 38,...

In [6]:
## 2. Generate the priority list for multiple sdf files with the default setting.

import os 

## get the sdf file name within the current directory 
sdf_ls = [f for f in os.listdir(os.getcwd()) if f.endswith('.sdf')]

## get priority list 
test2=confpass.conp(sdf_ls)
test2.get_priority()
test2.reidx_priority_df


name                                        priority_ls
0          test_15.sdf  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 20, 17, 24...
1  omegacsd_VORJER.sdf  [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...

In [7]:
## 3. Create a csv file for the priority list data frame in test 2 

test2.priority2csv()

o pipe_x_as_19082022.csv created


In [8]:
## 4.1. Create the gjf files for top 5% of the conformers in the priority list 

test1 = confpass.conp(['test_15.sdf'])
test1.get_priority()

keywords = '''%nprocshared=32
%mem=4GB
# opt freq wb97xd/6-31g(d) int=ultrafine

Title Card Required

0 1
'''

space='''
'''

test1.priority2gjf(keywords, space, 0.05)

In [9]:
## 4.2. Create the gjf files for a selection of conformers of your own choice 

conf_idx_ls=[[1,3]]
test1.get_gjf(keywords, space, conf_idx_ls)


## conpass.get_gjf() and conpass.priority2gjf() can also be applied on multiple sdf file 
## in these cases, the format of conf_idx_ls should be conf_idx_ls = [[1,3],[1,3] ...]

In [10]:
## 5. Processing radical molecules (with a different number of atoms compared to the pseudo structure): 
## Create the gjf files for top 5% of the conformers in the priority list 

## if the radical structure and the pseudo structure from conformational searching are different,  
## you will need to specify the removed atoms 
## i.e. removing the atom with index = 32 in the gjf file

test5 = confpass.conp(['./radical/radical_gu_liu1.sdf'])
test5.get_priority()

keywords = '''%nprocshared=32
%mem=4GB
# opt freq wb97xd/6-31g(d) int=ultrafine

Title Card Required

0 2
'''

space='''
'''

test5.priority2gjf(keywords, space, 0.05, radical = True, rmAtom_ls=[32])



In [11]:
## 6.	Generate the priority list for multiple sdf files with the every nth method, where n = 5

## get the sdf file name within the current directory 
sdf_ls = [f for f in os.listdir(os.getcwd()) if f.endswith('.sdf')]

## get priority list 
test3=confpass.conp(sdf_ls)
test3.get_priority(method ='nth', n=5)
test3.reidx_priority_df


name                                        priority_ls
0          test_15.sdf  [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 2, ...
1  omegacsd_VORJER.sdf  [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56,...

In [12]:
## 7.	Generate the priority list for multiple sdf files with the pipe_x method,
## where x = 0.7. Create a csv file for the priority list data frame. 


## get the sdf file name within the current directory 
sdf_ls = [f for f in os.listdir(os.getcwd()) if f.endswith('.sdf')]

## get priority list 
test4=confpass.conp(sdf_ls)
test4.get_priority(method ='pipe_x', x=0.7)
test4.priority2csv()
test4.reidx_priority_df



o pipe_x_19082022.csv created


name                                        priority_ls
0          test_15.sdf  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16...
1  omegacsd_VORJER.sdf  [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...

### PART 2: class pas (post-reoptimisations: predict the completeness of the reoptimisation process) 

## class confpass.pas

### __init__(self, path)

#### Parameters 

path : string; the path for the pas test; the path of the folder that contains the DFT output file from Gaussian16 calculations

### confpass.pas.preparation(self, molf='0.99', ra= False, rm_ls=[])

#### Parameters 

molf: string; default = '0.1'; choices=('0.1', '0.99'); changing the nor_nχ parameter for the test: nor_nχ=0.1 (looking for low energy conformers) or nor_nχ=0.99 (only looking for the global minimum)   

radical: bool; default = False; need to change to True when processing radicals - with a different number of atoms compared to the pseudo structure

rmAtom_ls: list; specify when you have removed atoms from the pseudo structure when generating the gjf file for reoptimisations; i.e. atom index starts from 1 


####  Attributes

delG_df: pd.DataFrame; energy data extracted from the Gaussian output files

g16sdf_name: string; the path of the structural information of the optimised structure in .sdf format

rmsCheck_cluster_df: pd.DataFrame; the result of the bins test workflow (i.e. rms calculations to pick out conformers with the same structures) 

model: sklearn model; trained random forest model 

molf: string; molf setting

### confpass.pas.make_prediction(self, p_x=0.8, p_x_de=0.2, p_n=3, p_method = 'pipe_x_as')

#### Parameters 

method: string; priority list assembling method; choices=('pipe_x_as', 'pipe_x','pipe_as','nth','random','ascend')

x: float; hyperparameter for the pipe_x method, default = 0.8; i.e. use the clustering result when n_clusters=total_conformer_no*x

x_de: float; hyperparameter for pipe_x_de method, default = 0.2; i.e. 20% (pipe x priority list) + 80%(pipe d priority list)

-n: float; hyperparameter for nth method, default = 3; i.e. prioritise every 3rd conformer

####  Attributes

label: int; 0 - reoptimisation incomplete; 1 - reoptimisation completed

prob_ratio: float; probability ratio - calculated from model.predict_log_proba()

confidence: string; high (probability ratio =< 0.3); low (probability ratio > 0.3)

## Examples

In [13]:
## 8.	Perform the pas test on a result folder. 
## The priority list is generated using the default setting.

path = os.getcwd()+'/test_15'

test6 = confpass.pas(path)
test6.preparation()
test6.make_prediction()

reoptimisation: complete; confidence level: 83.681
probability ratio: 0.095


In [14]:
## 9.	Perform the pas test on multiple result folders under the same directory. 
## Following example 8 and write a for loop 

In [15]:
## 10. Perform the pas test on a result folder. 
## Use the nor_nx=0.1 setting (only aiming to find the most stable conformer).
## The priority list is generated using the default setting.

test7 = confpass.pas(path)
test7.preparation(molf='0.1')
test7.make_prediction()


reoptimisation: incomplete; confidence level: 47.2
probability ratio: 0.891


In [18]:
## 11.	Perform the pas test on a result folder for a radical species. 
## The priority list is generated using the default setting 
## The programme is unable to cope with multiple radical molecules at the moment
## please also do not attempt to process a mix of radical and non radical species together 

## if the radical structure and the pseudo structure from conformational searching are different,  
## you will need to specify the removed atoms 


test8 = confpass.pas('./radical/radical_gu_liu1')
test8.preparation(ra=True, rm_ls=[32],molf='0.99')
test8.make_prediction()

reoptimisation: complete; confidence level: 68.263
probability ratio: 0.338


(double click this box if you think that the formatting is funny)


Format of result folder – your files need to follow the below naming format for the script to work:  

test_15/

├─ spe/
│   ├─ test_15_1_spe.out
│   ├─ test_15_2_spe.out
│   ├─ test_15_3_spe.out
│   ├─ ...
├─ test_15_1.out
├─ test_15_2.out
├─ test_15_3.out
├─ test_15.sdf <-- conformational searching output 
├─ ...

(opt+)freq output file: Name_idx.out 
Single point energy output file: Name_idx_spe.out 
